# First of all...remember the imports!!!

Ohhhh...Python is so easy and nice 'cause there's always someone else doing the hard stuff...

![Image](../images/import.jpg)

In [1]:
import requests
import pandas as pd
import re
import json
import math
import os
from dotenv import load_dotenv

# ...and remenber to leave everything nice and tidy...

![Image](../images/spaghetti.jpg)


In [2]:
load_dotenv("./.env")
TOKEN = os.environ.get("API_TOKEN")

In [3]:
API_TOKEN = TOKEN #API TOKEN (REMEMBER: do not push these to your repo) # Tocar
USERNAME = 'AndrewBavuels/' #USERNAME # Tocar
BASE_URL = 'https://api.github.com/'
KEY = 'repos/'
OWNER = 'ih-datapt-mad/'
REPO = 'dataptmad0923_labs/' # Tocar
SEARCH = 'search/issues?q=repo:'+OWNER+REPO+'+type:pr+state:{}'
PULLS = 'pulls?page={}&per_page=100&state={}'
COMMITS = 'pulls/{}/commits'
STATE = 'open'

In [4]:
BASE_URL + KEY + OWNER + REPO + PULLS

'https://api.github.com/repos/ih-datapt-mad/dataptmad0923_labs/pulls?page={}&per_page=100&state={}'

In [5]:
field_list1 = ['number',
               'title',
               'state',
               'created_at',
               'updated_at',
               'closed_at',
               'html_url',
               'base.repo.full_name',
               'base.ref',
               'head.repo.full_name',
               'head.ref',
               'head.repo.pushed_at']

In [6]:
field_list2 = ['student_name',
               'number',
               'lab_name',
               'state',
               'lab_status',
               'created_at',
               'updated_at',
               'closed_at',
               'html_url',
               'base.repo.full_name',
               'base.ref',
               'head.repo.full_name',
               'head.ref',
               'head.repo.pushed_at']

In [7]:
field_sort1 = ['lab_status',
               'lab_name',
               'student_name']

In [8]:
field_name1 = ['Student Name',
               'PR Number',
               'Lab Name',
               'PR Status',
               'Lab Status',
               'PR Created at',
               'PR Updated at',
               'PR Closed at',
               'PR URL',
               'base repository',
               'base',
               'head repository',
               'compare',
               'Pushed at']

# Functions 

Here you may find all functions you need to create your Pipeline. Be sure you test them before you start to build your Pipeline.

![Image](../images/pieces.jpg)

### Auxiliary Functions

In [9]:
# Aux Function 1: You can get only 100 results per page so it is important to know the number of pages you'll need.

def pages(base_url, search, state, username, api_token):
    pages = requests.get(base_url + search.format(state), auth=(username,api_token)).json()['total_count']
    if STATE == 'open':
        pages = math.ceil(pages/100)
        return pages
    elif STATE == 'closed':
        pages = math.ceil(pages/100)
        return pages

In [10]:
# Auc Function 2: Check the committs in order to know which labs are ready to be reviewed.

def get_commits(base_url, key, owner, repo, commits, pull, username, api_token):
    r_commits = requests.get(base_url + key + owner + repo + commits.format(pull),
                             auth=(username, api_token)).json()
    df_commits = pd.json_normalize(r_commits)
    list_commits = list(df_commits['commit.message'])
    commit = list(set([commit if commit == 'lab-finished' else 'lab-started' for commit in list_commits]))
    if 'lab-finished' in commit:
        return 'lab-finished'
    else:
        return 'lab-started'

In [11]:
# Aux Function 3: But the students aren't careful with the naming...

def student_name(x):
    if ']' in x:
        x = x.split(']')
        x = x[1].replace('_', ' ').strip()
        len_x = len(x.split(' '))
        if len_x > 1:
            x = re.findall('\w[a-zA-Z áéíóúÁÉÍÓÚñÑ-]+', x)
            x = x[0].strip()
            return x
        else:
            x = 'No student name provided'
            return x
    else:
        x = 'Pull request is not properly named'
        return x

In [12]:
#Aux Function 4

def lab_name(x):
    if ']' in x:
        x = x.split(']')
        x = x[0] + ']'
        x = x.strip()
        lower_case = re.findall('[A-ZÁÉÍÓÚñÑ]+', x)
        if x[0] == '[' and x[-1] == ']' and ' ' not in x and len(lower_case) == 0:
            return x
        else:
            x = 'Lab format name is incorrect'
            return x
    else:
        x = 'Pull request is not properly named'
        return x

In [13]:
# Aux Function 5: ...or forget to push their work!!!

def time_parser(x):
    try:
        x = x.strip()
        x = re.findall('[0-9]+', x)
        x = ''.join(x)
        x = pd.to_datetime(x, format='%Y%m%d%H%M%S', errors='coerce')
        return x
    except:
        return 'Nothing pushed yet'

### Pipeline Functions

In [14]:
# Pipeline Function 1: And finally get the 'pull requests'.

def get_pulls(base_url, key, owner, repo, pulls, search, state, username, api_token, field_list):
    pulls_list = []
    max_pages = pages(base_url, search, state, username, api_token)
    for i in range(max_pages):
        r_pulls = requests.get(base_url + key + owner + repo + pulls.format(i+1, state),
                               auth=(username, api_token)).json()
        df_pulls = pd.json_normalize(r_pulls)
        pulls_list.append(df_pulls)
    df_pulls = pd.concat(pulls_list)
    df_pulls = df_pulls[field_list]
    return df_pulls

In [15]:
# Pipeline Function 2: Apply!!!!!!

def df_status(df_pulls, base_url, key, owner, repo, commits, username, api_token, field_list):
    df_pulls['student_name'] = df_pulls['title'].apply(student_name)
    df_pulls['lab_name'] = df_pulls['title'].apply(lab_name)
    df_pulls['created_at'] = df_pulls['created_at'].apply(time_parser)
    df_pulls['updated_at'] = df_pulls['updated_at'].apply(time_parser)
    df_pulls['head.repo.pushed_at'] = df_pulls['head.repo.pushed_at'].apply(time_parser)
    df_pulls['lab_status'] = df_pulls.apply(lambda col: get_commits(base_url,
                                                                    key,
                                                                    owner,
                                                                    repo,
                                                                    commits,
                                                                    col['number'],
                                                                    username,
                                                                    api_token), axis=1)
    df_status = df_pulls[field_list]
    return df_status

In [16]:
# Pipeline function 3: And there you have it!!!

def create_csv(df_status, field_sort, field_name):
    df_csv = df_status.sort_values(by=field_sort, ascending=False)
    df_csv.columns = field_name
    df_csv.to_csv('../data/labs_status.csv', index=False)
    return df_csv

In [17]:
# Damned Pipelines!!!

DF_PULLS = get_pulls(BASE_URL, KEY, OWNER, REPO, PULLS, SEARCH, STATE, USERNAME, API_TOKEN, field_list1)
DF_STATUS = df_status(DF_PULLS, BASE_URL, KEY, OWNER, REPO, COMMITS, USERNAME, API_TOKEN, field_list2)
DF_CSV = create_csv(DF_STATUS, field_sort1, field_name1)
DF_CSV

,Student Name,PR Number,Lab Name,PR Status,Lab Status,PR Created at,PR Updated at,PR Closed at,PR URL,base repository,base,head repository,compare,Pushed at
21,Tomas Morin,202,[understanding-descriptive-stats],open,lab-started,2023-12-12 17:57:12,2023-12-12 17:57:12,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,tomasmorin97/dataptmad0923_labs,understanding-descriptive-stats,2024-02-05 13:41:22
23,Alana Castillo,189,[understanding-descriptive-stats],open,lab-started,2023-12-06 17:10:14,2023-12-12 19:09:35,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,AlanaCastillo/dataptmad0923_labs,understanding-descriptive-stats,2024-02-04 18:04:13
9,Alberto Castro,286,[resolving-git-conflicts],open,lab-started,2024-02-03 22:08:54,2024-02-03 22:08:54,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,castroalberto/dataptmad0923_labs,lab-resolving-git-conflicts,2024-02-04 16:10:18
0,Tomas Morin,295,[mysql],open,lab-started,2024-02-05 13:43:27,2024-02-05 13:43:27,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,tomasmorin97/dataptmad0923_labs,mysql,2024-02-05 13:41:22
24,Tomas Morin,187,[matplotlib-seaborn],open,lab-started,2023-12-05 12:57:25,2024-01-22 12:44:31,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,tomasmorin97/dataptmad0923_labs,matplotlib-seaborn,2024-02-05 13:41:22
33,Alana Castillo,73,[map-reduce_filter],open,lab-started,2023-10-07 13:11:16,2024-01-11 20:51:38,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,AlanaCastillo/dataptmad0923_labs,map-reduce-filter,2024-02-04 18:04:13
34,Natalia Vargas,69,[map-reduce-filter],open,lab-started,2023-10-07 13:03:06,2023-10-07 13:03:06,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,nataliavargas13/dataptmad0923_labs,map-reduce-filter,2023-10-07 13:00:04
26,No student name provided,171,[lab-web-scraping],open,lab-started,2023-11-07 12:16:27,2023-11-07 12:16:27,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,Guillemorato/dataptmad0923_labs,lab-web-scraping,2024-01-29 17:01:22
28,No student name provided,165,[lab-web-scraping],open,lab-started,2023-11-04 13:18:38,2024-02-03 22:32:34,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,castroalberto/dataptmad0923_labs,lab-web-scraping-alberto,2024-02-04 16:10:18
30,John Bryan,160,[lab-web-scraping],open,lab-started,2023-11-04 13:10:43,2024-01-21 11:23:45,None,https://github.com/ih-datapt-mad/dataptmad0923...,ih-datapt-mad/dataptmad0923_labs,main,JohnBryan02/dataptmad0923_labs,lab-web-scraping-johnbryan,2024-02-03 08:41:23
